# Homework 4
- Student: Aleksandra Kocot
 - [Git Hub page with the Homework Week4 questions](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2023/04-evaluation/homework.md)

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv

--2023-09-23 16:48:28--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73250 (72K) [text/plain]
Saving to: ‘AER_credit_card_data.csv’

AER_credit_card_dat 100%[===================>]  71,53K  --.-KB/s    in 0,04s   

2023-09-23 16:48:29 (1,58 MB/s) - ‘AER_credit_card_data.csv’ saved [73250/73250]



In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

##### The goal of this homework is to inspect the output of different evaluation metrics by creating a classification model (target column: card).

## Preparation
Create the target variable by mapping yes to 1 and no to 0.<br>
Split the dataset into 3 parts: train/validation/test with 60%/20%/20% distribution. Use train_test_split function for that with random_state=1.

In [5]:
df_raw = pd.read_csv("AER_credit_card_data.csv")

In [6]:
df_raw

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,yes,0,37.66667,4.5200,0.033270,124.983300,yes,no,3,54,1,12
1,yes,0,33.25000,2.4200,0.005217,9.854167,no,no,3,34,1,13
2,yes,0,33.66667,4.5000,0.004156,15.000000,yes,no,4,58,1,5
3,yes,0,30.50000,2.5400,0.065214,137.869200,no,no,0,25,1,7
4,yes,0,32.16667,9.7867,0.067051,546.503300,yes,no,2,64,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1314,yes,0,33.58333,4.5660,0.002146,7.333333,yes,no,0,94,1,19
1315,no,5,23.91667,3.1920,0.000376,0.000000,no,no,3,12,1,5
1316,yes,0,40.58333,4.6000,0.026513,101.298300,yes,no,2,1,1,2
1317,yes,0,32.83333,3.7000,0.008999,26.996670,no,yes,0,60,1,7


#### Target variable

In [8]:
df_raw["target"] = (df_raw["card"] == "yes").astype(int)

In [9]:
df_raw

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active,target
0,yes,0,37.66667,4.5200,0.033270,124.983300,yes,no,3,54,1,12,1
1,yes,0,33.25000,2.4200,0.005217,9.854167,no,no,3,34,1,13,1
2,yes,0,33.66667,4.5000,0.004156,15.000000,yes,no,4,58,1,5,1
3,yes,0,30.50000,2.5400,0.065214,137.869200,no,no,0,25,1,7,1
4,yes,0,32.16667,9.7867,0.067051,546.503300,yes,no,2,64,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,yes,0,33.58333,4.5660,0.002146,7.333333,yes,no,0,94,1,19,1
1315,no,5,23.91667,3.1920,0.000376,0.000000,no,no,3,12,1,5,0
1316,yes,0,40.58333,4.6000,0.026513,101.298300,yes,no,2,1,1,2,1
1317,yes,0,32.83333,3.7000,0.008999,26.996670,no,yes,0,60,1,7,1


#### Train, test, validation set split

In [12]:
df_full, df_test = train_test_split(df_raw, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

## Question 1
ROC AUC could also be used to evaluate feature importance of numerical variables.<br>

Let's do that<br>

For each numerical variable, use it as score and compute AUC with the card variable.<br>
Use the training dataset for that.<br>
If your AUC is < 0.5, invert this variable by putting "-" in front<br>

(e.g. -df_train['expenditure'])<br>

AUC can go below 0.5 if the variable is negatively correlated with the target varialble. <br>
You can change the direction of the correlation by negating this variable - then negative correlation becomes positive.<br>
<br>
Which numerical variable (among the following 4) has the highest AUC?
- reports
- dependents
- active
- share


In [13]:
df_train

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active,target
0,no,3,40.50000,4.0128,0.000299,0.00000,no,no,1,12,1,17,0
1,no,1,32.33333,6.0000,0.000200,0.00000,yes,no,4,18,1,4,0
2,yes,1,29.16667,2.2000,0.038205,69.79333,no,no,0,49,1,7,1
3,yes,1,54.66667,7.2900,0.106536,647.20670,yes,no,2,78,1,9,1
4,yes,0,25.00000,3.3984,0.000353,0.00000,yes,no,2,29,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,yes,0,53.00000,2.4500,0.017718,35.67500,yes,no,0,3,1,11,1
787,no,2,30.58333,2.5000,0.000480,0.00000,no,no,0,12,1,18,0
788,yes,0,24.75000,1.8750,0.080708,126.02250,no,no,1,14,0,1,1
789,yes,1,56.91667,3.4838,0.062895,182.09580,yes,no,0,540,1,7,1


In [35]:
np.correlate(df_train.target, df_train[c])

array([614])

In [54]:
aucs = {}
numeric_cols = [c for c in df_train.columns if c not in ["card", "owner", "selfemp"]]
for c in numeric_cols:
    feature_auc = roc_auc_score(df_train.target, df_train[c])
    feature_corr = np.corrcoef(df_train.target, df_train[c])
    if feature_auc < 0.5:
            feature_auc = roc_auc_score(df_train.target, -df_train[c])
    aucs[c] = [feature_auc, abs(feature_corr[0][1])]
    

In [55]:
results = pd.DataFrame(aucs).T
results.columns = ["AUC","COR_abs"]

In [56]:
results.sort_values(by="AUC", ascending=False).style.background_gradient(axis=0)  

,AUC,COR_abs
target,1.000000,1.000000
expenditure,0.991042,0.391337
share,0.989184,0.406993
reports,0.716663,0.448771
active,0.604317,0.107173
income,0.590805,0.102310
majorcards,0.534386,0.074889
dependents,0.532776,0.056133
months,0.529422,0.033637
age,0.524002,0.033578


Correlation shows linear relation while AUC detect also non-linear one so it may be useful for non-linear models.
- But logistic regression is linear underneath, so here it may be misleading (?)

#### ROC AUC

In [29]:
roc_auc_score([0,0,0,0,1,1,1,1], [0,0,0,0,1,1,1,1])

1.0

In [30]:
roc_auc_score([0,0,0,0,1,1,1,1], [0,1,0,1,0,1,0,1])

0.5

In [31]:
roc_auc_score([0,0,0,0,1,1,1,1], [10,20,30,40,50,60,70,80])

1.0

In [57]:
roc_auc_score([0,0,0,0,1,1,1,1], [10,20,30,40,45,46,47,48])

1.0

#### Correlation

In [61]:
np.corrcoef([0,0,0,0,1,1,1,1], [10,20,30,40,50,60,70,80])[0][1]

0.8728715609439693

In [59]:
np.corrcoef([0,0,0,0,1,1,1,1], [10,20,30,40,45,46,47,48])[0][1]

0.804195017233548

#### Training the model
From now on, use these columns only: <br>

["reports", "age", "income", "share", "expenditure", "dependents", "months", "majorcards", "active", "owner", "selfemp"]<br>
Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:<br>

LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)

## Question 2
What's the AUC of this model on the validation dataset? (round to 3 digits)
- 0.615
- 0.515
- 0.715
- 0.995